# Импорт библиотек

In [1]:
import re
import os

import pandas as pd
import numpy as np
import textstat
import optuna
import pysrt
import nltk
import ssl

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

## Запустить при первом запуске.

In [2]:
# ssl._create_default_https_context = ssl._create_unverified_context
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

# Открытие данных 

## Открытие excel и обработка

In [3]:
df = pd.read_excel('english_level/english_scores/movies_labels.xlsx').drop('id', axis = 1)
df.head()

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [4]:
df["Level"].value_counts()

B2            101
B1             55
C1             40
A2/A2+         26
B1, B2          8
A2              6
A2/A2+, B1      5
Name: Level, dtype: int64

Заменим промежуточные уровни на определенные, чтобы в итоге получить более равномерное распределение.

In [5]:
dictinory = {"A2/A2+": "A2", "B1, B2": "B1", "A2/A2+, B1": "B1"}
df["Level"] = df["Level"].replace(dictinory)

In [6]:
df[df.duplicated]

,Movie,Level
44,Inside_out(2015),B1
68,Powder(1995),B1
99,The_terminal(2004),B1


Удаляем дубликаты

In [7]:
df = df.drop_duplicates()

In [8]:
df.isna().sum()

Movie    0
Level    0
dtype: int64

## Открытие субтитров и обработка

In [9]:
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Z ]+[\)\]]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'

def clean_subs(subs):
    txt = re.sub(HTML, ' ', subs) # html тэги меняем на пробел
    txt = re.sub(TAG, ' ', txt) # тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты меняем на пробел
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы
    txt = ".".join(txt.lower().split('.')[1:-1]) # удаляем первый и последний субтитр (обычно это реклама)
    txt = re.sub(r"\n", "", txt)

    return txt

In [10]:
subs = pd.DataFrame({"Movie": [], "level_filepath":[], "sub": []})
root_directory = os.path.abspath("") + '/english_level/english_scores/Subtitles_all/'
extension = '.srt'


for root, dirs, files in os.walk(root_directory):
    for file_name in files:
        if file_name.endswith(extension):
            file_path = os.path.join(root, file_name)
            try: sub = clean_subs(pysrt.open(file_path).text)
            except:
                try: sub = clean_subs(pysrt.open(file_path, encoding='latin-1').text)
                except: sub = clean_subs(pysrt.open(file_path, encoding='cp1252').text)
            parts = file_path.split("/")
            subs = pd.concat([pd.Series({"Movie": ".".join(parts[-1].split(".")[:-1]), 
                                       "level_filepath":parts[-2], 
                                       "sub":sub}),
                            subs], ignore_index=True, axis = 1)
subs = subs.T.dropna()
subs.head()

,Movie,level_filepath,sub
0,Aladdin(1992),Subtitles,"please, please, come closer.too close. a littl..."
1,The_Intern(2015),Subtitles,"work and love.that's all there is.""well, i'm r..."
2,The_Fundamentals_of_Caring(2016),Subtitles,it is also about understandinghow to navigatea...
3,Her(2013),Subtitles,"opensubtitles.org todaywww.titlovi.com""to my c..."
4,Matilda(1996),Subtitles,some will grow to be butchers orbakers or ca...


## Соединение

In [11]:
df = df.merge(subs, on = 'Movie', how = 'outer')

In [12]:
df.isna().sum()

Movie              0
Level             48
level_filepath     7
sub                7
dtype: int64

Обобщим уровни в переменные "Level" и "level_filepath". Затем заменить пропущенные значения в переменной "Level" имеющимися данными из "level_filepath". 

Удалить фильмы, у которых уровень указан как "Subtitles".

In [13]:
df['Level'] = df['Level'].fillna(df['level_filepath'])
df = df[df['Level'] != "Subtitles"]

In [14]:
df.isna().sum()

Movie             0
Level             0
level_filepath    7
sub               7
dtype: int64

Теперь можно убрать level_filepath


In [15]:
df = df.drop('level_filepath', axis = 1)

В данных присутствует 7 пропусков, которые требуется заполнить. Поскольку имеется очень мало данных, оценка уровня фильмов, отсутствующих в этих данных, требует глубокого анализа произведения, что усложняет их добавление из стороних источников.

In [16]:
root_directory = os.path.abspath("") + '/english_level/other_movie'

for root, dirs, files in os.walk(root_directory):
    for file_name in files:
        if file_name.endswith(extension):
            file_path = os.path.join(root, file_name)
            try: sub = clean_subs(pysrt.open(file_path).text)
            except:
                try: sub = clean_subs(pysrt.open(file_path, encoding='latin-1').text)
                except: sub = clean_subs(pysrt.open(file_path, encoding='cp1252').text)
            parts = file_path.split("/")
            movie_name = ".".join(parts[-1].split(".")[:-1])
            df.loc[df["Movie"] == movie_name, 'sub'] = sub
            if movie_name == "Thor love and thunder":
                df.loc[df["Movie"] == "Thor: love and thunder", 'sub'] = sub

In [17]:
df

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"enjoy the flick ben on phone: michelle, p..."
1,10_things_I_hate_about_you(1999),B1,"so, cameron. here you go.nine schools in 1 0 y..."
2,A_knights_tale(2001),B2,two minutes or forfeit.lend us those.right. le...
3,A_star_is_born(2018),B2,black eyes open wideit's time to testify t...
4,Aladdin(1992),A2,"please, please, come closer.too close. a littl..."
...,...,...,...
281,"Crown, The S01E03 - Windsor.en",B2,-are you hearing me? -hear you clearly. stan...
282,"Crown, The S01E05 - Smoke and Mirrors.en",B2,"ah, there you are. come in.i'm practicing for ..."
283,"Crown, The S01E04 - Act of God.en.SDH",B2,-[philip] fuel on.chocks are in position. swit...
284,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,B2,"you were right.it's time for me to go home.oh,..."


# Добавление новых признаков и дополнительная обработка

In [18]:
STOP_WORDS = set(stopwords.words('english') + ['uh', 'oh', 'hmm', 'huh', 'ha', 'oooh'])
STEMMER = WordNetLemmatizer()
LETTERS = r'[^a-zA-Z\'.,!? ]'


def text_stats(row):
    row['smog_index'] = textstat.smog_index(row['sub'])
    row['coleman_liau_index'] = textstat.coleman_liau_index(row['sub'])
    row["dale_chall_readability_score"] = textstat.dale_chall_readability_score(row['sub'])
    row["gunning_fog"] = textstat.gunning_fog(row['sub'])
    
    txt = row['sub']
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(r"[^\w\s]", "", txt)
    txt = word_tokenize(txt)
    txt = " ".join([STEMMER.lemmatize(word) for word in txt if word.lower() not in STOP_WORDS])
    
    row['sub'] = txt
    
    return row

df = df.apply(text_stats, axis = 1)

In [19]:
keys = {"B2": 3,
        "B1": 2,
        "C1": 4,
        "A2": 1}

df['Level'] = df['Level'].map(keys)

In [20]:
train, test = train_test_split(df, test_size=0.1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# Подбор гипер параметров

In [21]:
# def objective(trial: optuna.trial.Trial):
#     train_local = train.copy
#     params = {
#         'iterations': trial.suggest_int("iterations", 100, 1000),
#         'learning_rate': trial.suggest_float("learning_rate", 1e-2, 1e-1, log=True),
#         'depth': trial.suggest_int("depth", 4, 10),
#         'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
#         'random_strength': trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
#         'od_type': trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
#         'od_wait': trial.suggest_int("od_wait", 10, 50),
#         'random_seed': 1234,
#         'eval_metric': "MultiClass",
#         'verbose': False,
#         "loss_function":'MultiClass',
#         "auto_class_weights": 'Balanced'}

#     vectorizer_param = {
#         "max_features":trial.suggest_int("max_features", 100, 2000),
#         "min_df":trial.suggest_int("min_df", 0, 100),
#         "max_df":trial.suggest_float("max_df", 0.5, 1, log=True)}
    
#     Vectorizer = TfidfVectorizer(**vectorizer_param, stop_words=None)
#     train_local = train.merge(pd.DataFrame(Vectorizer.fit_transform(train['sub']).toarray()), 
#                         left_index=True, 
#                         right_index=True)
#     train_local = train_local.drop(['sub', 'Movie'], axis = 1)
    
#     features = train_local.drop("Level", axis = 1)
#     target = train_local['Level']
    
#     results = []
#     random_seed = np.random.RandomState(1234)

#     kf = KFold(n_splits=3, shuffle=True, random_state=1234)
#     for train_index, test_index in kf.split(features):

#         features_train, features_test = features.iloc[train_index], features.iloc[test_index]
#         target_train, target_test = target.iloc[train_index], target.iloc[test_index]

#         features_train, features_valid, target_train, target_valid = train_test_split(features_train,
#                                                                                       target_train,
#                                                                                       test_size=0.1,
#                                                                                       random_state=random_seed)
#         model = CatBoostClassifier(**params)
#         model.fit(features_train, target_train,
#                   use_best_model=True,
#                   eval_set=(features_valid,
#                             target_valid),
#                   early_stopping_rounds=50)
        
#         prediction = model.predict(features_test)
#         results.append(balanced_accuracy_score(target_test, prediction))
        
#     print(params)
#     print()
#     print(vectorizer_param)
#     return np.mean(results)


# studies = optuna.create_study(direction='maximize', study_name="CatBoost")
# studies.optimize(objective, n_trials=1000)
# best_catboost_params = studies.best_params | {'random_seed': 1234, 'verbose': False,
#                                               'eval_metric': "BalancedAccuracy", }
# studies.best_value

Для простоты пропустим подбор гиперпараметров.


Полученные в ходе подбора гипер параметры:

{'iterations': 719, 'learning_rate': 0.015748110341147425, 'depth': 5, 'l2_leaf_reg': 18.974532325902207, 'random_strength': 2.9674758606838096e-07, 'od_type': 'IncToDec', 'od_wait': 35, 'random_seed': 1234, 'eval_metric': 'MultiClass', 'verbose': False, 'loss_function': 'MultiClass', 'auto_class_weights': 'Balanced'}

{'max_features': 1921, 'min_df': 83, 'max_df': 0.9191182527705366}

# Создание финальной модели и выводы.

In [22]:
cat_boost_param = {'iterations': 719, 'learning_rate': 0.015748110341147425, 
                   'depth': 5, 'l2_leaf_reg': 18.974532325902207, 
                   'random_strength': 2.9674758606838096e-07, 'od_type': 'IncToDec', 
                   'od_wait': 35, 'random_seed': 1234, 'eval_metric': 'MultiClass', 
                   'verbose': False, 'loss_function': 'MultiClass', 
                   'auto_class_weights': 'Balanced'}

vectorizer_param = {'max_features': 3000, 'min_df': 83, 'max_df': 0.9191182527705366}

In [23]:
cat_boost = CatBoostClassifier(**cat_boost_param)
vectorizer = TfidfVectorizer(**vectorizer_param, stop_words=None)

In [24]:
def get_X_and_y(dataframe):
    
    dataframe = dataframe.drop(['sub', 'Movie'], axis = 1)
    
    X = dataframe.drop("Level", axis = 1)
    y = dataframe['Level']
    
    return X, y

vectorizer.fit(train['sub'])
train = train.merge(pd.DataFrame(vectorizer.transform(train['sub']).toarray()), 
                left_index=True, 
                right_index=True)
test = test.merge(pd.DataFrame(vectorizer.transform(test['sub']).toarray()), 
                left_index=True, 
                right_index=True)

In [25]:
features_train, target_train = get_X_and_y(train)
features_test, target_test = get_X_and_y(test)

In [26]:
features_train, features_valid, target_train, target_valid = train_test_split(features_train,
                                                                              target_train,
                                                                              test_size=0.1,
                                                                              random_state=1234)
cat_boost.fit(features_train, target_train,
             use_best_model=True,
                  eval_set=(features_valid,
                            target_valid),
                  early_stopping_rounds=50)

In [27]:
print("Качетсво модели", balanced_accuracy_score(target_test, cat_boost.predict(features_test)))

0.71875

**Выводы**

В ходе нашей работы мы провели исследование английских фильмов, разделенных на различные уровни сложности. Одной из важных задач было собрать недостающие субтитры и подготовить их для использования в моделях машинного обучения.

В процессе анализа предоставленных фильмов мы обнаружили существенный дисбаланс в уровнях сложности. Особенно заметно было недостаточное количество фильмов с категорией A2, а также наличие промежуточных категорий. Учитывая недостаток данных, мы приняли решение исключить промежуточные категории.

В результате наших усилий нам удалось создать модель с balanced accuracy, достигающей уровня 0.7187. Однако, для дальнейшего улучшения качества работы, мы настоятельно рекомендуем предоставить большее количество размеченных фильмов. Это позволит улучшить точность и обобщающую способность модели, обеспечивая более надежные результаты.